# 1.Creating Model

In [19]:
from keras import layers
from tensorflow.keras.models import Sequential
#%pip install keras-tuner --upgrade
import keras_tuner as kt   
from tensorflow.keras import optimizers


In [20]:
def build_model(hp):
    model = Sequential()

    model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape = (150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))


    model.add(layers.Flatten())

    model.add(layers.Dense(units=hp.Choice("layer size", [128, 256, 512]),activation="relu")) #64, 128, 256, 512
    model.add(layers.Dense(1,activation="sigmoid"))
    
    model.compile(loss = "binary_crossentropy",
                  optimizer = optimizers.RMSprop(learning_rate = 1e-4), #ogrenme orani
                  metrics = ["acc"]
              )

    return model

In [ ]:
keras_tuner = kt.RandomSearch(build_model,
                              objective="val_acc",
                              overwrite = True,
                              max_trials = 3,
                              directory = "models/parameter_tuning",
                              project_name = "parameter_tuning"
                              )

In [22]:
train_directory = "dataset/catdog/train"
validation_directory = "dataset/catdog/validation"

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_directory(train_directory,
                                                    target_size = (150,150),
                                                    batch_size = 20,
                                                    class_mode = "binary"
                                                    )

validation_generator = train_datagen.flow_from_directory(validation_directory,
                                                        target_size = (150,150),
                                                        batch_size = 20,
                                                        class_mode = "binary"
                                                    )

Found 4567 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.


In [ ]:
keras_tuner.search_space_summary()
keras_tuner.search(train_generator, validation_data = validation_generator, epochs = 25)
best_model = keras_tuner.get_best_models()[0]

229/229 ━━━━━━━━━━━━━━━━━━━━ 145s 629ms/step - acc: 0.7785 - loss: 0.5503 - val_acc: 0.5000 - val_loss: 0.9725
Epoch 2/25
229/229 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - acc: 0.7803 - loss: 0.5234

In [ ]:
best_model = keras_tuner.get_best_models()[0]